In [1]:
import json
import zipfile
import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
import math
import csv
import operator
import Geohash
from build import TreeBuilding
from decision import haversine
from sklearn.cross_validation import train_test_split
import pickle

%matplotlib inline

In [2]:
air_trajs = pd.read_csv('air_trajs.csv',
                        #index_col=,
                        usecols = ['POLYLINE', 'Destination'],
                        converters={'POLYLINE': lambda x: json.loads(x), 'Destination': lambda x: json.loads(x)})

In [3]:
trajectories = list(air_trajs['POLYLINE'])
targets = list(air_trajs['Destination'])

In [4]:
X_train, X_test, y_train, y_test = train_test_split(trajectories, targets, train_size=0.70, random_state = 0)

In [5]:
tb = TreeBuilding()
tb = tb.fit(X_train, y_train, 1, 10)

length 3029
('len', 3029)
length 2202
('len', 2202)
length 1630
('len', 1630)
length 1424
('len', 1424)
length 951
('len', 951)
length 840
('len', 840)
length 745
('len', 745)
length 657
('len', 657)
length 612
('len', 612)
length 522
('len', 522)
length 437
('len', 437)
length 290
('len', 290)
length 243
('len', 243)
length 218
('len', 218)
length 192
('len', 192)
length 170
('len', 170)
length 139
('len', 139)
length 94
('len', 94)
length 59
('len', 59)
length 20
('len', 20)
length 18
('len', 18)
length 15
('len', 15)
length 11
('len', 11)
length 7
length 4
length 4
length 3
length 1
length 39
('len', 39)
length 11
('len', 11)
length 9
length 1
length 27
('len', 27)
length 10
length 17
('len', 17)
length 9
length 8
length 32
('len', 32)
length 24
('len', 24)
length 8
length 16
('len', 16)
length 3
length 13
('len', 13)
length 9
length 4
length 8
length 45
('len', 45)
length 23
('len', 23)
length 13
('len', 13)
length 8
length 5
length 10
length 22
('len', 22)
length 9
length 13
('len

In [6]:
def predict_batch(traj_tree, trajectories):
    return [predict_one(traj_tree, trajectory) for trajectory in trajectories] 
    
def predict_one(traj_tree, trajectory):
    current_node = traj_tree.root
    
    for idx, point in enumerate(trajectory):
        if len(trajectory) > (idx + 1):
            if (current_node.left == None) and (current_node.right == None):
                prediction = current_node.prediction
                
            elif (current_node.left == None) and (current_node.right != None):
                current_node = current_node.right
                
            elif (current_node.left != None) and (current_node.right == None):
                current_node = current_node.left
                
            else:
                if haversine(point, current_node.decision_point) < current_node.radius:
                    current_node = current_node.left
                else:
                    current_node = current_node.right
                    
        else:
            prediction = current_node.prediction
            
    #return [float(prediction[0]), float(prediction[1])]
    return prediction

In [7]:
def prediction_error(y_val, y_hat):
    return np.mean(np.power([haversine(y_val[i], y_hat[i]) for i in xrange(len(y_val))], 2))

In [6]:
y_hat = tb.predict(tb, X_test)

AttributeError: Tree instance has no attribute 'predict'

In [70]:
prediction_error(y_test, y_hat)

523.71657286206573

In [10]:
def mean_points_alpha(points):
    lats = [point[0] for point in points]
    lons = [point[1] for point in points]
    return [np.mean(lats), np.mean(lons)]

def sqare_error_alpha(points):
    pmean = mean_points_alpha(points)
    return np.mean(np.power([haversine(point, pmean) for point in points], 2))

In [65]:
sqare_error_alpha(y_test)

492.11715109521219

In [50]:
print tb.root.decision_point

[ -8.666676  41.233914]


In [8]:
pickle.dump(tb, open( "tree_10_new.p", "wb" ))

In [10]:
load_tree = pickle.load( open( "tree_10_new.p", "rb" ))

In [11]:
y_hat = predict_batch(load_tree, X_test)
prediction_error(y_test, y_hat)

500.95632688532623

In [ ]:
import gmplot

gmap = gmplot.GoogleMapPlotter(41.1496100, -8.6109900 , 12)

#gmap.plot(heat_lats, heat_lons, 'cornflowerblue', edge_width=10)
#gmap.scatter(heat_lats, heat_lons, '#3B0B39', size=10, marker=False)
gmap.scatter(start_lats, start_lons, '#006400', size=10, marker=False)
gmap.scatter(end_lats, end_lons, '#8B0000', size=10, marker=False)

#gmap.heatmap(heat_lats, heat_lons)

gmap.draw("mymap.html")